#Variational Quantum Eigensolver(VQE)、QAOAセミナー 2-4（組合せ最適化問題編2）

素因数分解を最小値問題に置き換えて解いてみます。


###Quantum factorization of 56153 with only 4 qubits
Nikesh S. Dattani,1,2,∗ Nathaniel Bryans3,†
1Quantum Chemistry Laboratory, Kyoto University, 606-8502, Kyoto, Japan, 2Physical & Theoretical Chemistry Laboratory,
Oxford University, OX1 3QZ, Oxford, UK, 3University of Calgary, T2N 4N1, Calgary, Canada. ∗
dattani.nike@gmail.com,
†
nbryans1@gmail.com

https://arxiv.org/pdf/1411.6758.pdf

###Variational Quantum Factoring
Eric R. Anschuetz, Jonathan P. Olson, Alán Aspuru-Guzik, Yudong Cao  
(Submitted on 27 Aug 2018)  
https://arxiv.org/abs/1808.08927  

###Prime Factorization Using Quantum Annealing and Algebraic Geometry
https://1qbit.com/wp-content/uploads/2016/09/1QBit-Research-Paper-%E2%80%93-Prime-Factorization-Using-Quantum-Annealing-and-Algebraic-Geometry.pdf

##具体的手順
手順はとてもシンプルです。これまでゲート方式はShorのアルゴリズムを使って問題を解いていましたが、そうではなくて最小値問題に落とし込みます。効率的な落とし込みかたがあり、前処理をいれて143の素因数分解は4量子ビットだけで解けます。

１、量子ビットを使って二進数で乗算を考える
２、桁数の特性を生かしてQUBO問題を簡略化する
３、単純化されたQUBO問題をQAOAアルゴリズムにかけて最小基底状態を求める。

##１、量子ビットを使って二進数の乗算（143の素因数分解）
まず二進数の乗算を考えます。二進数の乗算と加算を学ぶ必要がありますが、乗算は筆算では10進数とほとんど同じです。

![11.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/1650e6ed-2e63-2bc7-977c-fb20da265c5e.png)

これは、m=p*qを考えますが、pを二進数で考えて、一番下と一番上の位は1になることはわかっています。一番上が1は自明で、素数なので一番下の位は0ではない（偶数ではない）です。pとqを二進数で表すと上記のようになります。

まずはこれを単純にかけます。$2^0$の位から10進数のようにかけて、ずらして足し合わせてきます。途中carriesというのは桁上がりになります。二進数で1+1をすると桁が上がるので、$Z_{12}$は$2^1$の位から$2^2$の位に上がると言う意味です。

##２、桁数の特性を生かしてイジング問題を簡略化する次に各位を足し合わせます。足し合わせると連立方程式は、

$$
\begin{eqnarray}p_1+q_1 &=& 1+2z_{12}\\ p_2+p_1q_1+q_2+z_{12} &=& 1+2z_{23}+4z_{24}\\ 1+p_2q_1+p_1q_2+1+z_{23} &=& 1+2z_{34}+4z_{35}\\ … q_2+p_2+z_{45}+z_{35} &=& 0+2z_{56}+4z_{57}\\ 1+z_{56}+z_{46} &=& 0+2z_{67}\\ z_{67}+z_{57} &=& 1\end{eqnarray}
$$

となります。ここで上から見ていくと、、、まず$p_1+q_1=1+2z_{12}$において、１が確定しているため、$p_1$と$q_1$はどちらかが1であるため、2の位はでてこないため、$z_{12}=0$がわかります。

同様に、$p_2+p_1q_1+q_2+z_{12} = 1+2z_{23}+4z_{24}$はまず、上記より$p_1$と$q_1$はどちらかが1でどちらかが0であるため、$p_1q_1=0$がわかる。また、$z_{12}=0$だったため、$p_2+q_2 = 1+2z_{23}+4z_{24}$となる。ここで、前述と同様1が確定しているため、$z_{23}=0,z_{24}=0$がわかります。

また、$1+p_2q_1+p_1q_2+1+z_{23} = 1+2z_{34}+4z_{35}$は、$z_{23}=0$より、$1+p_2q_1+p_1q_2 = 2z_{34}+4z_{35}$となる。まず桁数が合わないので$z_{35}=0$となり、$1+p_2q_1+p_1q_2 = 2z_{34}$ここで、左辺に1があるため、$z_{34}$は0でないので1が確定する。

これらを進めていってまとめると、下記の式が得られます。

$$
\begin{eqnarray}p_1+q_1-1 &=&0\\
p_2+q_2-1 &=& 0\\
p_2q_1+p_1q_2-1 &=&0
\end{eqnarray}
$$

この３つの連立方程式を最小値問題に落とすためには、各方程式を二乗すると必ず0の時に最小になるので、連立方程式から下記のハミルトニアンを得ます。

$$
H=(p_1+q_1-1)^2 + (p_2+q_2-1)^2+(p_2q_1+p_1q_2-1)^2\\=5-3p_1-p_2-q_1+2p_1q_1-3p_2q_1+2p_1p_2q_1-3q_2+p_1q_2+2p_2q_2+2p_2q_1q_2
$$

これを確認のために、総当たりでコスト計算してみると、バイナリ値で$\mid 0000 \rangle$から$\mid 1111\rangle$までで、

$5,2,4,1,4,3,0,1,2,0,3,1,1,1,1,3$

となり、$\mid 6 \rangle$と$\mid 9 \rangle$の時に$H=0$となるので、先に対応する、$\mid 0110 \rangle$と$\mid 1001 \rangle$が答えとなり、

$p=13,q=11$と$p=11,q=13$が答えとなります。


##３、問題をQAOAアルゴリズムにかけて最小基底状態を求める。

実際にQAOAを使って正しいか確認します。

In [1]:
!pip3 install blueqat

     |████████████████████████████████| 51kB 1.5MB/s 


In [2]:
from blueqat import vqe 
from blueqat.pauli import qubo_bit as q 

hamiltonian = -3*q(0)-q(1)-q(2)+2*q(0)*q(2)-3*q(1)*q(2)+2*q(0)*q(1)*q(2)-3*q(3)+q(0)*q(3)+2*q(1)*q(3)+2*q(1)*q(2)*q(3) 
step = 4 

result = vqe.Vqe(vqe.QaoaAnsatz(hamiltonian, step)).run() 
print(result.most_common(5)) 

(((1, 1, 0, 0), 0.2184939468894782), ((0, 0, 1, 1), 0.2184939468894781), ((1, 0, 0, 1), 0.18033782434124815), ((0, 1, 1, 0), 0.1474805194922803), ((1, 1, 1, 0), 0.04239566140768896))


ここで、きちんと答えがえら得ていて、0110と1001が十分な確率振幅で得られている。このため量子ゲート方式で基底状態が求まり、素因数分解ができました。

ちなみに量子アニーリング方式でも解くことができるが、この場合には上記にある3量子ビットの多体問題の2体問題への分解が必要となる。量子ゲートでは3量子ビットの相互作用をハミルトニアンに入れたまま計算できるので、量子ゲートの方が前処理がとても簡単に済むと言う利点があります。